In [13]:
# Load required libraries
 # For rmvnorm
library(aaltobda)  # For bioassay example functions
      # For Pareto-k diagnostics



# Load the bioassay data
data("bioassay")   # assuming the package provides this

# Step 1: Draw 4000 samples from the prior (bivariate normal)
set.seed(123)
S <- 4000
prior_mean <- c(0, 0)
prior_cov <- diag(2)
theta_samples <- rmvnorm(S, mean = prior_mean, sigma = prior_cov)

# Step 2: Calculate log-likelihood for each sample
log_likelihood <- apply(theta_samples, 1, function(theta) {
  sum(dbinom(
    bioassay$y, bioassay$n, 
    plogis(bioassay$x * theta[1] + theta[2]), 
    log = TRUE
  ))
})

# Step 3: Compute normalized importance weights
log_weights <- log_likelihood - max(log_likelihood)  # stabilize
weights <- exp(log_weights)
weights <- weights / sum(weights)                    # normalize

# Step 4: Importance sampling estimate of posterior mean
posterior_mean <- colSums(theta_samples * weights)
alpha_est <- posterior_mean[1]
beta_est <- posterior_mean[2]

cat("Posterior mean estimates:\n")
cat("alpha =", alpha_est, "\n")
cat("beta  =", beta_est, "\n")

# Step 5: Compute Effective Sample Size (ESS)
ESS <- 1 / sum(weights^2)
cat("Effective Sample Size (ESS) =", ESS, "\n")

# Step 6: Monte Carlo Standard Error (MCSE)
mcse_alpha <- sqrt(sum(weights * (theta_samples[,1] - alpha_est)^2)) / sqrt(ESS)
mcse_beta  <- sqrt(sum(weights * (theta_samples[,2] - beta_est)^2)) / sqrt(ESS)
cat("MCSE:\n")
cat("alpha =", mcse_alpha, "\n")
cat("beta  =", mcse_beta, "\n")

# Step 7: Pareto-k diagnostic
# Use 'weights' as log ratios (loo package expects log weights)
pareto_k <- posterior::pareto_khat(x = log_weights, are_log_weights = TRUE)
#pareto_k <- posterior::pareto_khat(theta_samples)
cat("Pareto-k diagnostic =", round(pareto_k, 1), "\n")

# Step 8: Trustworthiness based on Pareto-k
if (pareto_k < 0.7) {
  cat("Importance sampling estimate is likely trustworthy.\n")
} else {
  cat("Importance sampling estimate may not be trustworthy.\n")
}


Posterior mean estimates:
alpha = 1.777175 
beta  = -0.09415046 
Effective Sample Size (ESS) = 257.6649 
MCSE:
alpha = 0.04027104 
beta  = 0.02888834 
Pareto-k diagnostic = 0.1 
Importance sampling estimate is likely trustworthy.
